In [77]:
# Imports
import numpy as np
import altair as alt
import pydot
from IPython.display import display, SVG, clear_output
import plotly.express as px

from pydrake.all import (
    AddMultibodyPlantSceneGraph, AngleAxis, BasicVector, 
    DiagramBuilder, FindResourceOrThrow, Integrator, JacobianWrtVariable, 
    LeafSystem, MeshcatVisualizerCpp, MultibodyPlant, MultibodyPositionToGeometryPose, Parser,
    PiecewisePose, PiecewisePolynomial, Quaternion, RigidTransform, 
    RollPitchYaw, RotationMatrix, SceneGraph, Simulator, TrajectorySource, MeshcatVisualizerParams
)
from pydrake.examples.manipulation_station import ManipulationStation

from manipulation import running_as_notebook
from manipulation.meshcat_cpp_utils import (
    StartMeshcat, MeshcatJointSlidersThatPublish)
from manipulation.scenarios import AddMultibodyTriad, SetColor

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

In [112]:
# Adding a table

builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
model = Parser(plant, scene_graph).AddModelFromFile('./diva_teleop.urdf', 'diva_robot')


# Transform for the table location
# X_R = RigidTransform(RotationMatrix.MakeYRotation(-np.pi/2), np.array([-0.1, 0.5, 1]))
X_R = RigidTransform(RotationMatrix(RollPitchYaw([0.8, -0.8, 0])), np.array([-0.1, 0.5, 1]))
Parser(plant, scene_graph).AddModelFromFile('./table/extra_heavy_duty_table_modified.sdf','table')

plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(), X_R)
# plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(table)[0]).body_frame(), X_T)
plant.Finalize()

# Delete any previous models
meshcat.Delete()
meshcat.DeleteAddedControls()

visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant.SetPositions(plant.GetMyContextFromRoot(context),model,
                      [0.21, -1.03, 1.28, -1.1, 2.8, 1.5, 0])
# To actually see the context, publish diagram
diagram.Publish(context)

In [114]:
def gripper_forward_kinematics_example():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    
    model = Parser(plant, scene_graph).AddModelFromFile('./diva_teleop.urdf', 'diva_robot')
    # Transform for the robot root
    X_R = RigidTransform(RotationMatrix.MakeYRotation(-np.pi/2), np.array([-0.1, 0.5, 1]))
    # Robot root transform similar to the berkley blue
    # X_R = RigidTransform(RotationMatrix(RollPitchYaw([-0.61087, -0.5236, 3.1416])), np.array([-0.1, 0.5, 1]))
    # Robot root transform similar to a bi-manual humanoid robot
#     X_R = RigidTransform(RotationMatrix(RollPitchYaw([0.61, -0.61, 0])), np.array([-0.1, 0.5, 1]))
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(), X_R)

    table = Parser(plant, scene_graph).AddModelFromFile('./table/extra_heavy_duty_table_modified.sdf','table')
    plant.Finalize()

    # Draw the frames
#     for body_name in ["base_link", "link1", "link2", "link3", "link4", "link5", "link6", "link7", "eef"]:
    for body_name in ["base_link", "eef"]:
        AddMultibodyTriad(plant.GetFrameByName(body_name), scene_graph)

    meshcat.Delete()
    meshcat.DeleteAddedControls()

    visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant.SetPositions(plant.GetMyContextFromRoot(context),model,
                      [0.21, -1.03, 1.28, -1.1, 2.8, 1.5, 0])
    
    gripper = plant.GetBodyByName("eef")
    def pose_callback(context):
        pose = plant.EvalBodyPoseInWorld(context, gripper)   ## This is the important line
        clear_output(wait=True)
        print("gripper position (m): " + np.array2string(
            pose.translation(), formatter={
                'float': lambda x: "{:3.2f}".format(x)}))
        print("gripper roll-pitch-yaw (rad):" + np.array2string(
            RollPitchYaw(pose.rotation()).vector(),
                         formatter={'float': lambda x: "{:3.2f}".format(x)}))

    sliders = MeshcatJointSlidersThatPublish(meshcat, plant, visualizer, context)
    # sliders.Run()
    sliders.Run(pose_callback)

gripper_forward_kinematics_example()

gripper position (m): [-0.38 0.07 0.82]
gripper roll-pitch-yaw (rad):[1.51 -0.14 1.65]
